In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from PIL import Image


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
EPOCH = 240
BATCH_SIZE = 128
LR = 0.1
device = torch.device("cuda")
x_train = np.load("/content/gdrive/My Drive/PRhw/x_train.npy")
y_train = np.load("/content/gdrive/My Drive/PRhw/y_train.npy")

x_test = np.load("/content/gdrive/My Drive/PRhw/x_test.npy")
y_test = np.load("/content/gdrive/My Drive/PRhw/y_test.npy")


In [4]:
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

print(x_train.shape)
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
print(np.unique(y_train))


50000 train samples
10000 test samples
(50000, 32, 32, 3)
[0 1 2 3 4 5 6 7 8 9]


In [5]:
class PROCESS(Dataset):
    def __init__(self, x, y, if_train):
        assert len(x) == len(y)
        self.data = x
        self.label = y
        self.train = if_train

    def __getitem__(self, index):
        if (self.train):
            transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
        else:
            transform = transforms.Compose([transforms.ToTensor(),
                                            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
        single_data = self.data[index]
        single_image = Image.fromarray(single_data)
        single_label = self.label[index]
        return transform(single_image), single_label

    def __len__(self):
        return len(self.data)


In [6]:
training_set = PROCESS(x_train, y_train, if_train=True)

# %%
print(Image.fromarray(x_train[0]))

# %%
testing_set = PROCESS(x_test, y_test, if_train=False)

trainloader = torch.utils.data.DataLoader(training_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testing_set, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')  # %%


<PIL.Image.Image image mode=RGB size=32x32 at 0x7F2C63B40780>


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

net = torchvision.models.resnet152(num_classes=10)
net = net.cuda()


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9,
                      weight_decay=1e-4)
# %%
best_acc = 85
print("Start Training, Resnet-152!")
pre_epoch = 0
# net.load_state_dict(torch.load('/content/gdrive/My Drive/PRhw/model/net_186.pth'))
with open("/content/gdrive/My Drive/PRhw/result/acc.txt", "w") as f:
    with open("/content/gdrive/My Drive/PRhw/result/log.txt", "w")as f2:
        for epoch in range(pre_epoch, EPOCH):
            if epoch >= 136:
                LR = 0.01
                optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=1e-4)
            if epoch >= 186:
                LR = 0.001
                optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=1e-4)
            print('\nEpoch: %d' % (epoch + 1))
            print("learning rate: ", LR)
            net.train()
            sum_loss = 0.0
            correct = 0.0
            total = 0.0
            for i, data in enumerate(trainloader, 0):

                length = len(trainloader)
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                labels = labels.reshape((labels.shape[0]))
                optimizer.zero_grad()

                # forward + backward
                outputs = net(inputs)
                #print(outputs.shape)
                #print(labels.shape)
                loss = criterion(outputs, labels)
                #print("loss: ", loss)
                loss.backward()
                optimizer.step()

                sum_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += predicted.eq(labels.data).cpu().sum()
                if (i+1+epoch*length) % 200 == 0:
                    print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
                          % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
                f2.write('%03d  %05d |Loss: %.03f | Acc: %.3f%% '
                         % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
                f2.write('\n')
                f2.flush()

            print("Waiting Test!")
            with torch.no_grad():
                correct = 0
                total = 0
                for data in testloader:
                    net.eval()
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    labels = labels.reshape((labels.shape[0]))
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum()
                print('test accuracy: %.3f%%' % (100 * correct / total))
                acc = 100. * correct / total
                print('Saving model......')
                torch.save(net.state_dict(),
                           '%s/net_%03d.pth' % ('/content/gdrive/My Drive/PRhw/model', epoch + 1))
                f.write("EPOCH=%03d,Accuracy= %.3f%%" % (epoch + 1, acc))
                f.write('\n')
                f.flush()
                if acc > best_acc:
                    f3 = open("/content/gdrive/My Drive/PRhw/result/best_acc.txt", "w")
                    f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch + 1, acc))
                    f3.close()
                    best_acc = acc
        print("Training Finished, TotalEPOCH=%d" % EPOCH)
# %%



Start Training, Resnet-50!

Epoch: 1
learning rate:  0.1
[epoch:1, iter:200] Loss: 2.369 | Acc: 10.062% 
Waiting Test!
test accuracy: 10.000%
Saving model......

Epoch: 2
learning rate:  0.1
[epoch:2, iter:400] Loss: 2.310 | Acc: 10.330% 
[epoch:2, iter:600] Loss: 2.305 | Acc: 10.104% 
Waiting Test!
test accuracy: 10.000%
Saving model......

Epoch: 3
learning rate:  0.1
[epoch:3, iter:800] Loss: 2.297 | Acc: 9.983% 
[epoch:3, iter:1000] Loss: 2.298 | Acc: 10.500% 
Waiting Test!
test accuracy: 10.000%
Saving model......

Epoch: 4
learning rate:  0.1
[epoch:4, iter:1200] Loss: 2.296 | Acc: 9.809% 
[epoch:4, iter:1400] Loss: 2.297 | Acc: 9.981% 
Waiting Test!
test accuracy: 11.000%
Saving model......

Epoch: 5
learning rate:  0.1
[epoch:5, iter:1600] Loss: 2.293 | Acc: 10.851% 
[epoch:5, iter:1800] Loss: 2.296 | Acc: 10.322% 
Waiting Test!
test accuracy: 11.000%
Saving model......

Epoch: 6
learning rate:  0.1
[epoch:6, iter:2000] Loss: 2.300 | Acc: 10.382% 
[epoch:6, iter:2200] Loss: 2.2

In [8]:
import torch.nn as nn
import torch.nn.functional as F

net = torchvision.models.resnet152(num_classes=10)
net = net.cuda()
net.load_state_dict(torch.load('/content/gdrive/My Drive/PRhw/model/net_240.pth'))

<All keys matched successfully>

In [46]:
from sklearn.metrics import accuracy_score
y_pred = []
print("Waiting Test!")
with torch.no_grad():
    correct = 0
    total = 0
    for data in testloader:
        net.eval()
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        labels = labels.reshape((labels.shape[0]))
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        y_pred += predicted.tolist()
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('test accuracy: %.3f%%' % (100. * correct / total))
    acc = 100. * correct / total
y_pred = np.array(y_pred)


Waiting Test!
test accuracy: 87.240%


In [47]:
# ## DO NOT MODIFY CODE BELOW!
# please screen shot your results and post it on your report
assert y_pred.shape == (10000,)
y_test = np.load("y_test.npy")
print("Accuracy of my model on test-set: ", accuracy_score(y_test, y_pred))


Accuracy of my model on test-set:  0.8724
